In [1]:
%matplotlib notebook
import pickle
import matplotlib.pyplot as plt

## Load INDRA graph and curations

In [2]:
with open('/Users/ben/data/indra_network.pkl', 'rb') as fh:
    g = pickle.load(fh)

In [3]:
from indra_db.client.principal.curation import get_curations
curs = get_curations()
curs_by_hash = defaultdict(set)
for cur in curs:
    curs_by_hash[cur['pa_hash']].add(cur['tag'])
incorrect_hashes = {stmt_hash for stmt_hash, tags in curs_by_hash.items()
                    if not tags & {'correct', 'hypothesis', 'activity_amount'}}

## Functions to rank nodes by evidence

In [4]:
def get_nodes_by_evidence(edge_data, namespaces=None):
    if not namespaces:
        namespaces = ['HGNC', 'FPLX']
    nodes = defaultdict(int)
    for node, edges_data in edge_data.items():
        if g.nodes[node]['ns'] not in namespaces:
            continue
        for edge in edges_data.values():
            if edge['stmt_type'] != 'Complex' and \
                    edge['stmt_hash'] not in incorrect_hashes:
                nodes[node] += edge['evidence_count']
    return dict(nodes)


def aggregate_evidences(*nodes_with_evidences, saturate=100):
    nodes = defaultdict(int)
    for nodes_with_evidences_one in nodes_with_evidences:
        for node, cnt in nodes_with_evidences_one.items():
            nodes[node] += min(cnt, saturate)
    return dict(nodes)

def evidence_count_list(targets, *nodes_with_evidences):
    nodes = defaultdict(list)
    for target, nodes_with_evidences_one in zip(targets, nodes_with_evidences):
        for node, cnt in nodes_with_evidences_one.items():
            nodes[node].append((target, cnt))
    return dict(nodes)


def get_children_by_evidence(node, namespaces=None):
    nodes_by_ev = get_nodes_by_evidence(g[node], namespaces=namespaces)
    nodes_by_ev = {k: v for k, v in nodes_by_ev.items()
                   if k != node}
    return nodes_by_ev


def get_parents_by_evidence(node, namespaces=None):
    incoming_edges = g.in_edges(node, data=True, keys=True)
    edges = defaultdict(dict)
    for parent, _, edge_id, data in incoming_edges:
        edges[parent][edge_id] = data
    return get_nodes_by_evidence(edges, namespaces=namespaces)


def get_top(counts_dict, n=None, exclude=None):
    sorted_list = sorted(counts_dict.items(), key=lambda x: x[1], reverse=True)
    if exclude:
        sorted_list = [(x, y) for x, y in sorted_list if x not in exclude]
    if n:
        sorted_list = sorted_list[:n]
    return sorted_list


def hindex(scores):
    for h in range(len(scores), 0, -1):
        if len([s for s in scores if s >= h]) >= h:
            return h
    return 1

def plot_interactors(entity, ev_count_list, interactor_type):
    interactors, scores = zip(*sorted(ev_count_list,
                                      key=lambda x: x[1], reverse=True))
    plt.figure()
    plt.title('Top %ss of %s' % (interactor_type, entity))
    plt.bar(range(len(interactors)), scores)
    plt.xticks(range(len(interactors)), labels=interactors, rotation=90)
    plt.savefig('%s_interactors.png' % entity, bbox_inches='tight')

## Load TAS inhibitor data

In [5]:
import pandas
url = ('https://raw.githubusercontent.com/samuelbunga/panacea_indra/nextflow/'
       'panacea_exam/drug_target_search/output/tas_indra_db_hits.csv')
df1 = pandas.read_csv(url, index_col=0)
drug_targets = {}
for _, row in df1.iterrows():
    name = row['Compound Name'].split(' ')[0]
    drug_targets[name] = sorted(row['tas_targets'].split(', '))
all_drugs = sorted(list(drug_targets.keys()))
len(all_drugs)

16

## Load kinomescan data for preliminary hits

In [6]:
df = pandas.read_csv('../data/210809-kinomescan-100nM-10uM-panacea-prelim-hits.csv')
df = df[df['Compound Concentration (nM)'] == 100]
threshold = 7
df = df[df['Percent Control'] < threshold]
kinomescan_targets = defaultdict(set)
kinomescan_target_scores = defaultdict(dict)
mappings = {'MST4': 'STK26'}
for _, row in df.iterrows():
    name = row['Compound Name'].split(' ')[0]
    if name == 'PD':
        name = 'PD 407824'
    discoverx_gene = row['DiscoveRx Gene Symbol']
    if '(' in discoverx_gene and ('JAK' not in discoverx_gene
            and 'TYK' not in discoverx_gene):
        continue
    gene = row['Entrez Gene Symbol']
    if gene in mappings:
        gene = mappings[gene]
    kinomescan_targets[name].add(gene)
    kinomescan_target_scores[name][gene] = (100-int(row['Percent Control']))/100

In [7]:
kinomescan_target_scores['Lestaurtinib']

{'AAK1': 0.96,
 'NUAK1': 0.97,
 'AURKA': 1.0,
 'AURKB': 1.0,
 'AURKC': 0.99,
 'CDKL5': 0.96,
 'CHEK1': 1.0,
 'CLK1': 0.94,
 'CLK4': 0.96,
 'STK17A': 1.0,
 'STK17B': 1.0,
 'MAPK15': 0.98,
 'GRK7': 0.99,
 'CHUK': 0.95,
 'IRAK1': 0.98,
 'IRAK3': 0.96,
 'IRAK4': 1.0,
 'JAK2': 1.0,
 'JAK3': 1.0,
 'MAPK8': 0.98,
 'MAPK10': 0.98,
 'LATS2': 1.0,
 'LRRK2': 0.98,
 'MAP3K2': 1.0,
 'MAP3K3': 0.98,
 'MAP4K2': 0.98,
 'MAP2K1': 1.0,
 'MAP2K2': 1.0,
 'MAP2K3': 0.99,
 'MAP2K4': 0.97,
 'MAP2K5': 0.97,
 'MELK': 1.0,
 'MINK1': 0.99,
 'MKNK2': 1.0,
 'MYLK3': 0.96,
 'MAP3K9': 0.98,
 'STK3': 1.0,
 'PHKG1': 0.98,
 'PHKG2': 0.95,
 'PKN2': 1.0,
 'PLK4': 0.98,
 'PRKD2': 0.98,
 'PRKD3': 0.94,
 'PRKG2': 1.0,
 'RIOK3': 0.94,
 'DSTYK': 0.99,
 'SGK1': 1.0,
 'SGK3': 0.99,
 'NUAK2': 1.0,
 'MAP3K7': 0.96,
 'TNK2': 0.94,
 'NTRK1': 1.0,
 'NTRK2': 1.0,
 'NTRK3': 0.99,
 'TYK2': 1.0,
 'ULK3': 0.99,
 'MAP3K19': 1.0,
 'ZAP70': 0.97}

## Load kinomescan data for all OKL

In [8]:
df = pandas.read_csv('../data/184-sample-set-US073-0016937-O_HAR001-01-p-00001-000-00_scanMAX_Data Report.csv')
df = df[df['Compound Concentration (nM)'] == 10000]
threshold = 85
df = df[df['Percent Control'] < threshold]
kinomescan_okl_targets = defaultdict(set)
kinomescan_okl_target_scores = defaultdict(dict)
mappings = {'MST4': 'STK26'}
for _, row in df.iterrows():
    hmslid = row['Compound Name']
    discoverx_gene = row['DiscoveRx Gene Symbol']
    if '(' in discoverx_gene and ('JAK' not in discoverx_gene
            and 'TYK' not in discoverx_gene):
        continue
    gene = row['Entrez Gene Symbol']
    if gene in mappings:
        gene = mappings[gene]
    kinomescan_okl_targets[hmslid].add(gene)
    kinomescan_okl_target_scores[hmslid][gene] = (100-int(row['Percent Control']))/100

## Load and process drug treatment data and get negative drugs

In [21]:
import numpy
plates = ['3963', '3964', '3965']
sheets = {plate: pandas.read_excel('../data/vp_1601_3963-3965_Summary.xlsx', sheet_name=plate)
          for plate in plates}
treatment_cols = ['X well Ratio Neurite 72H']
dmso_col = ['D well Ratio Neurite 72H']
treatment_vals = defaultdict(list)
for plate, df in sheets.items():
    print(plate)
    dmso_df = df[~pandas.isna(df['Plate']) & ~pandas.isna(df['D well Ratio Neurite 72H_A'])]
    avg_a = numpy.mean(dmso_df['D well Ratio Neurite 72H_A'])
    avg_b = numpy.mean(dmso_df['D well Ratio Neurite 72H_B'])
    dmso_avg = numpy.mean([avg_a, avg_b])
    treatment_df = df[~pandas.isna(df['Plate']) & ~pandas.isna(df['X well Ratio Neurite 72H_A'])]
    for _, row in treatment_df.iterrows():
        treatment_avg = numpy.mean([row['X well Ratio Neurite 72H_A'],
                                    row['X well Ratio Neurite 72H_B']])
        #print(treatment_avg, row['Compound Name'])
        change = (treatment_avg - dmso_avg)/dmso_avg
        treatment_vals[row['Compound Name']].append(change)
        #if numpy.abs(treatment_avg - dmso_avg)/dmso_avg < 0.05:
            

3963
3964
3965


In [23]:
sorted(treatment_vals)

['(+)-JQ1 (S)-JQ1 HMSL10328',
 '(S)-Crizotinib HMSL10577',
 '1-Azakenpaullone 1-Akp Azakenpaullone HMSL12145',
 'A 1070722 HMSL12137',
 'A-674563 HMSL11465',
 'AEE788 NVP-AEE788 (R)-6-(4-((4-Ethylpiperazin-1-yl)methyl)phenyl)-N-(1-phenylethyl)-7H-pyrrolo[2 3-d]pyrimidin-4-amine HMSL11021',
 'AMG-208 HMSL11363',
 'AMG-548 HMSL12021',
 'AST 487 HMSL12073',
 'AT-7519 HMSL10004',
 'AT9283 HMSL10393',
 'AZD 5438 KIN001-239 HMSL10158',
 'AZD-1480 HMSL10139',
 'AZD8055 HMSL10007',
 'Afatinib BIBW-2992 HMSL10133',
 'Alectinib CH5424802 HMSL10201',
 'Alisertib MLN8237 HMSL10391',
 'Alsterpaullone 9-Nitropaullone NSC 705701 HMSL12138',
 'Alvocidib Flavopiridol HMR-1275 L868275 HMSL10011',
 'Apitolisib RG7422 GDC-0980 HMSL10234',
 'Axitinib AG013736 HMSL10443',
 'BAY61-3606 HMSL10166',
 'BGJ398 KIN001-271 NVP-BGJ398 HMSL10183',
 'BI-2536 NPK33-1-98-1 HMSL10041',
 'BIX 02565 HMSL10434',
 'BMS 777607 HMSL10143',
 'BMS-599626 AC480 HMSL11476',
 'BMS-754807 HMSL10290',
 'BMS-911543 HMSL12077',
 'BMS3

In [10]:
num_bottom_drugs = 32
bottom_hits_with_scores = sorted(treatment_vals.items(), key=lambda x: max([abs(y) for y in x[1]]),
                                 reverse=False)[:num_bottom_drugs]
bottom_hits_hmsl = [hit[0].split(' ')[-1] for hit in bottom_hits_with_scores]

In [11]:
bottom_hits_hmsl

['HMSL12155',
 'HMSL10007',
 'HMSL10426',
 'HMSL10037',
 'HMSL12148',
 'HMSL10051',
 'HMSL10018',
 'HMSL10048',
 'HMSL11588',
 'HMSL12059',
 'HMSL10099',
 'HMSL10226',
 'HMSL11476',
 'HMSL10072',
 'HMSL10066',
 'HMSL12145',
 'HMSL10023',
 'HMSL10182',
 'HMSL10168',
 'HMSL10114',
 'HMSL10442',
 'HMSL10056',
 'HMSL10010',
 'HMSL10685',
 'HMSL12078',
 'HMSL12067',
 'HMSL10167',
 'HMSL10156',
 'HMSL12077',
 'HMSL10633',
 'HMSL11643',
 'HMSL12139']

## Process drug-target relations and target ranking

In [12]:
ndrugs = len(kinomescan_target_scores)

In [13]:
target_aggregate_score = defaultdict(list)
for drug, target_scores in kinomescan_target_scores.items():
    for target, score in target_scores.items():
        target_aggregate_score[target].append(score)
        
bottom_target_aggregate_score = defaultdict(list)
for drug in bottom_hits_hmsl:
    target_scores = kinomescan_okl_target_scores[drug]
    for target, score in target_scores.items():
        bottom_target_aggregate_score[target].append(score)

In [19]:
bottom_target_aggregate_score['MAPK8']

[0.99,
 0.51,
 0.63,
 0.34,
 1.0,
 0.19,
 0.34,
 0.53,
 1.0,
 0.35,
 0.17,
 1.0,
 0.42,
 0.2,
 0.98,
 1.0,
 0.4,
 0.89,
 0.56,
 1.0,
 0.7,
 0.5,
 1.0,
 0.89]

In [15]:
def target_auc(target_aggregate_score, ndrugs, plot_species=None):
    thresholds = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    target_scores = {}
    for target, scores in target_aggregate_score.items():
        values = [len([s for s in scores if s > thresh])
                  for thresh in thresholds]
        score = sum(values) / (ndrugs * len(thresholds))
        target_scores[target] = score
        if plot_species and target in plot_species:
            plt.figure()
            plt.plot(thresholds, values)
            plt.title(target)
            plt.ylabel('Number of drugs inhibiting %s at a given level' % target)
            plt.xlabel('Level of inhibition')
            plt.ylim([0, ndrugs])
            plt.show()
            plt.savefig('%s.png' % target)
    return target_scores

In [16]:
target_aucs = target_auc(target_aggregate_score, ndrugs=ndrugs)
bottom_target_aucs = target_auc(bottom_target_aggregate_score, ndrugs=num_bottom_drugs)
top_overall_targets_scores = get_top(target_aucs, 30)
bottom_overall_target_scores = get_top(bottom_target_aucs, 30)
top_overall_targets_scores

[('MAP3K19', 0.46153846153846156),
 ('STK17A', 0.3076923076923077),
 ('STK17B', 0.3076923076923077),
 ('MAPK15', 0.23076923076923078),
 ('JAK2', 0.23076923076923078),
 ('JAK3', 0.23076923076923078),
 ('TYK2', 0.23076923076923078),
 ('AURKC', 0.23076923076923078),
 ('MAP2K5', 0.23076923076923078),
 ('BLK', 0.15384615384615385),
 ('KIT', 0.15384615384615385),
 ('STK10', 0.15384615384615385),
 ('PDGFRB', 0.15384615384615385),
 ('CHUK', 0.15384615384615385),
 ('IRAK1', 0.15384615384615385),
 ('ROCK2', 0.15384615384615385),
 ('ULK3', 0.15384615384615385),
 ('PRKCE', 0.15384615384615385),
 ('BMP2K', 0.15384615384615385),
 ('MAP4K2', 0.15384615384615385),
 ('MYLK3', 0.15384615384615385),
 ('RIOK3', 0.15384615384615385),
 ('NTRK1', 0.15384615384615385),
 ('AURKB', 0.15384615384615385),
 ('CLK1', 0.15384615384615385),
 ('CLK4', 0.15384615384615385),
 ('MAPK8', 0.15384615384615385),
 ('MAPK10', 0.15384615384615385),
 ('MINK1', 0.15384615384615385),
 ('MAP3K9', 0.15384615384615385)]

In [17]:
get_top(bottom_target_aucs)

[('ALK', 0.578125),
 ('MAP2K5', 0.575),
 ('MKNK2', 0.571875),
 ('PDGFRB', 0.5375),
 ('PIP4K2C', 0.534375),
 ('PRKCE', 0.528125),
 ('MKNK1', 0.525),
 ('CSF1R', 0.515625),
 ('MAPK10', 0.5125),
 ('MAPK8', 0.50625),
 ('MAPK9', 0.50625),
 ('TYK2', 0.503125),
 ('MAP3K19', 0.5),
 ('PIK3C3', 0.484375),
 ('DCLK3', 0.48125),
 ('BUB1', 0.478125),
 ('MTOR', 0.478125),
 ('PRKD1', 0.471875),
 ('PI4KB', 0.46875),
 ('MAP4K1', 0.4625),
 ('DYRK2', 0.45625),
 ('FGFR3', 0.45625),
 ('LCK', 0.453125),
 ('JAK3', 0.446875),
 ('BTK', 0.4375),
 ('DDR1', 0.4375),
 ('VRK2', 0.4375),
 ('CSNK1G2', 0.434375),
 ('STK10', 0.434375),
 ('MERTK', 0.428125),
 ('ERN1', 0.421875),
 ('MATK', 0.41875),
 ('IRAK3', 0.415625),
 ('SBK1', 0.409375),
 ('AXL', 0.403125),
 ('GSG2', 0.4),
 ('PRKD3', 0.4),
 ('NEK10', 0.396875),
 ('RIOK2', 0.396875),
 ('RIOK3', 0.39375),
 ('PDGFRA', 0.390625),
 ('STK17A', 0.384375),
 ('STK25', 0.384375),
 ('ERBB3', 0.378125),
 ('HIPK4', 0.378125),
 ('IKBKE', 0.378125),
 ('TIE1', 0.378125),
 ('NTRK1', 0.

In [18]:
target_aucs['JAK3'], target_aucs['CSNK1A1']

KeyError: 'CSNK1A1'

In [ ]:
plt.figure()
plt.bar(range(len(top_overall_targets_scores)), [s for _, s in top_overall_targets_scores])
plt.xticks(range(len(top_overall_targets_scores)),
           labels=[t for t, _ in top_overall_targets_scores],
          rotation=90)
plt.yticks([0, 0.2, 0.4, 0.6, 0.8, 1])
plt.savefig('drug_targets_bar.png', bbox_inches='tight')

plt.figure()
plt.bar(range(len(bottom_overall_target_scores)), [s for _, s in bottom_overall_target_scores])
plt.xticks(range(len(bottom_overall_target_scores)),
           labels=[t for t, _ in bottom_overall_target_scores],
          rotation=90)
plt.yticks([0, 0.2, 0.4, 0.6, 0.8, 1])
plt.savefig('bottom_drug_targets_bar.png', bbox_inches='tight')

In [ ]:
top_overall_targets = [t for t, _ in top_overall_targets_scores]
bottom_overall_targets = [t for t, _ in bottom_overall_target_scores]
top_overall_targets

In [ ]:
rank_diff = {}
for top_rank, top_target in enumerate(top_overall_targets):
    if top_target in bottom_overall_targets:
        bottom_rank = bottom_overall_targets.index(top_target)
    else:
        bottom_rank = 30
    rank_diff[top_target] = bottom_rank - top_rank
sorted_rank_diff = get_top(rank_diff)

plt.figure()
plt.bar(range(len(sorted_diff)), [s for _, s in sorted_diff])
plt.xticks(range(len(sorted_diff)),
           labels=[t for t, _ in sorted_diff],
          rotation=90)
plt.savefig('sorted_rank_diff.png', bbox_inches='tight')

### FIXME
score_diff = {}
bottom_target_score_dict = dict(bottom)
for top_target, top_score in top_overall_targets_scores:
    if top_target in bottom_overall_target:
        bottom_rank = bottom_overall_targets.index(top_target)
    else:
        bottom_rank = 30
    score_diff[top_target] = bottom_rank - top_rank
sorted_rank_diff = get_top(score_diff)
### FIXME

plt.figure()
plt.bar(range(len(sorted_diff)), [s for _, s in sorted_diff])
plt.xticks(range(len(sorted_diff)),
           labels=[t for t, _ in sorted_diff],
          rotation=90)
#plt.yticks([0, 0.2, 0.4, 0.6, 0.8, 1])
plt.savefig('sorted_score_diff.png', bbox_inches='tight')

## Rank downstreams of drug targets

In [ ]:
children_by_evidence = [get_children_by_evidence(t, namespaces=['HGNC', 'FPLX'])
                        for t in top_overall_targets]
ev_count_list = evidence_count_list(top_overall_targets, *children_by_evidence)
overall_downstreams = aggregate_evidences(*children_by_evidence)
top_overall_downstreams = get_top(overall_downstreams, 25)
get_top(overall_downstreams)

In [ ]:
bottom_children_by_evidence = [get_children_by_evidence(t, namespaces=['HGNC', 'FPLX'])
                                for t in bottom_overall_targets]
bottom_ev_count_list = evidence_count_list(bottom_overall_targets, *bottom_children_by_evidence)
bottom_overall_downstreams = aggregate_evidences(*bottom_children_by_evidence)
top_bottom_overall_downstreams = get_top(bottom_overall_downstreams, 25)
get_top(bottom_overall_downstreams)

In [ ]:
def plot_top_downstreams(score_dict, method):
    interactors, scores = zip(*sorted(score_dict,
                                  key=lambda x: x[1], reverse=True))
    plt.figure()
    plt.title('Top downstream proteins')
    plt.bar(range(len(interactors)), scores)
    plt.xticks(range(len(interactors)), labels=interactors, rotation=90)
    plt.savefig('top_downstreams_%s.png' % method, bbox_inches='tight')

In [ ]:
plot_top_downstreams(get_top(overall_downstreams, 20), 'ev_count')
plot_top_downstreams(get_top(bottom_overall_downstreams, 20), 'ev_count')

In [ ]:
ev_count_list['STAT3']

In [ ]:
for protein in ['MINK1', 'MAP2K5', 'JAK3', 'TNIK', 'GAK', 'CIT', 'MAP4K4', 'MAP4K2', 'MAP4K1', 'STK10',
                'MAP3K7']:
    plot_interactors(protein, get_top(get_children_by_evidence(protein), 10),
                     'downstream target')

plot_interactors('STAT3', ev_count_list['STAT3'], 'regulator')
plot_interactors('NFkappaB', ev_count_list['NFkappaB'], 'regulator')
plot_interactors('ERK', ev_count_list['ERK'], 'regulator')
plot_interactors('JNK', ev_count_list['JNK'], 'regulator')

In [ ]:
ev_count_list['NFkappaB']

In [ ]:
get_top(get_children_by_evidence('MAP2K5', namespaces=['HGNC', 'FPLX']))

In [ ]:
hindexes = {downstream: hindex([x for _, x in count_list])
            for downstream, count_list in ev_count_list.items()}
get_top(hindexes, 20)

In [ ]:
bottom_hindexes = {downstream: hindex([x for _, x in count_list])
                   for downstream, count_list in bottom_ev_count_list.items()}
get_top(bottom_hindexes, 20)

In [ ]:
plot_top_downstreams(get_top(hindexes, 20), 'hindex')

In [ ]:
top_overall_downstreams

In [ ]:
network_edges = []
# Add drug-target edges from kinomescan
for drug, targets in kinomescan_targets.items():
    for target in targets:
        if target in top_overall_targets:
            network_edges.append((drug, target,
                                  {'penwidth': kinomescan_target_scores[drug][target]}))
# Add downstreams of targets
for target in top_overall_targets:
    children = get_children_by_evidence(target)
    for child, num_ev in children.items():
        if child in [x for x, _ in top_overall_downstreams] and num_ev > 2:
             network_edges.append((target, child, {'penwidth': num_ev/10}))
network_edges

In [ ]:
len(network_edges)

In [ ]:
import networkx
G = networkx.DiGraph()
G.add_edges_from([edge for edge in network_edges if 'TNIK' in edge or 'MINK1' in edge])
def draw(graph):
    ag = networkx.nx_agraph.to_agraph(graph)
    # Add some visual styles to the graph
    #ag.node_attr['shape'] = 'plaintext'
    ag.graph_attr['splines'] = True
    ag.graph_attr['rankdir'] = 'LR'
    ag.draw('graph.png', prog='dot')
    ag.draw('graph.pdf', prog='dot')
    return ag
ag = draw(G)
from IPython.display import Image
Image("graph.png")

Drug specific results

In [ ]:
downstreams = {}
downstream_genes = {}
downstream_processes = {}
for drug, targets in drug_targets.items():
    downstreams[drug] = aggregate_evidences(
        *[get_children_by_evidence(t) for t in targets])
    downstream_genes[drug] = aggregate_evidences(
        *[get_children_by_evidence(t, namespaces=['HGNC']) for t in targets])
    downstream_processes[drug] = aggregate_evidences(
        *[get_children_by_evidence(t, namespaces=['MESH', 'GO']) for t in targets])

In [ ]:
get_children_by_evidence('MAPK9', namespaces=['HGNC', 'FPLX'])

In [ ]:
df = pandas.read_csv('../data/kwx_tx_6hr_TF.csv')
df

In [ ]:
top_inhibited_tfs = ['ATF4', 'FOXM1', 'MEF2C', 'WT1', 'MBD1']
tf_parents_by_evidence = [get_parents_by_evidence(t, namespaces=['HGNC', 'FPLX'])
                        for t in top_inhibited_tfs]
tf_ev_count_list = evidence_count_list(top_overall_targets, *tf_parents_by_evidence)
overall_tf_upstreams = aggregate_evidences(*tf_parents_by_evidence)

In [ ]:
get_top(overall_tf_upstreams)

# Multilinear regression

In [65]:
import numpy
plates = ['3963', '3964', '3965']
sheets = {plate: pandas.read_excel('../data/vp_1601_3963-3965_Summary.xlsx', sheet_name=plate)
          for plate in plates}
treatment_cols = ['X well Ratio Neurite 72H']
dmso_col = ['D well Ratio Neurite 72H']
treatment_vals = defaultdict(list)
for plate, df in sheets.items():
    print(plate)
    dmso_df = df[~pandas.isna(df['Plate']) & ~pandas.isna(df['D well Ratio Neurite 72H_A'])]
    avg_a = numpy.mean(dmso_df['D well Ratio Neurite 72H_A'])
    avg_b = numpy.mean(dmso_df['D well Ratio Neurite 72H_B'])
    dmso_avg = numpy.mean([avg_a, avg_b])
    treatment_df = df[~pandas.isna(df['Plate']) & ~pandas.isna(df['X well Ratio Neurite 72H_A'])
                      & (df['Molar Concentration'] == '10 uM')]
    for _, row in treatment_df.iterrows():
        treatment_avg = numpy.mean([row['X well Ratio Neurite 72H_A'],
                                    row['X well Ratio Neurite 72H_B']])
        change = (treatment_avg - dmso_avg)/dmso_avg
        compound_hmslid = row['Compound Name'].split(' ')[-1]
        treatment_vals[compound_hmslid] = change
treatment_vals = dict(treatment_vals)

3963
3964
3965


In [66]:
sorted(treatment_vals.items(), key=lambda x: x[1])

[('HMSL10378', -0.5439491862662458),
 ('HMSL10150', -0.49446902968780543),
 ('HMSL10020', -0.4213527460798385),
 ('HMSL10032', -0.33883237068169697),
 ('HMSL12140', -0.31217049250309714),
 ('HMSL12153', -0.29467034443347917),
 ('HMSL10290', -0.27928989339827903),
 ('HMSL10120', -0.2730273347909054),
 ('HMSL11487', -0.27116692686916727),
 ('HMSL10206', -0.2650073807680125),
 ('HMSL12143', -0.261581772442719),
 ('HMSL12080', -0.2512599905220905),
 ('HMSL10128', -0.2302031419576916),
 ('HMSL10494', -0.22572219351306322),
 ('HMSL10380', -0.2212028810659286),
 ('HMSL10231', -0.21103093692652927),
 ('HMSL10073', -0.20573755165231386),
 ('HMSL10141', -0.20333231322855128),
 ('HMSL10149', -0.20290379329422104),
 ('HMSL10143', -0.1976721903317863),
 ('HMSL10068', -0.18699977889179203),
 ('HMSL12076', -0.1861410407631826),
 ('HMSL12081', -0.18596586967176232),
 ('HMSL10394', -0.18453419403040103),
 ('HMSL10207', -0.17927975441561378),
 ('HMSL10327', -0.17106412739155422),
 ('HMSL12152', -0.16788

In [67]:
df = pandas.read_csv('../data/184-sample-set-US073-0016937-O_HAR001-01-p-00001-000-00_scanMAX_Data Report.csv')
df = df[df['Compound Concentration (nM)'] == 10000]
kinomescan_okl_targets = defaultdict(set)
kinomescan_okl_target_scores = defaultdict(dict)
mappings = {'MST4': 'STK26'}
for _, row in df.iterrows():
    hmslid = row['Compound Name']
    discoverx_gene = row['DiscoveRx Gene Symbol']
    if '(' in discoverx_gene and ('JAK' not in discoverx_gene
            and 'TYK' not in discoverx_gene):
        continue
    gene = row['Entrez Gene Symbol']
    if gene in mappings:
        gene = mappings[gene]
    kinomescan_okl_target_scores[hmslid][gene] = (100-int(row['Percent Control']))/100

In [68]:
kinomescan_data = numpy.array(
    [[kinomescan_okl_target_scores[hmslid][gene]
      for gene in sorted(kinomescan_okl_target_scores[hmslid])]
     for hmslid in sorted(kinomescan_okl_target_scores)])

In [98]:
training_data = kinomescan_data[:100, :]
test_data = kinomescan_data[100:, :]

In [99]:
training_data.shape

(100, 385)

In [100]:
sorted_hmslids = sorted(kinomescan_okl_target_scores)
sorted_genes = sorted(kinomescan_okl_target_scores[hmslid])

In [102]:
response = numpy.array([treatment_vals.get(hmslid, 0) for hmslid in sorted_hmslids],)
training_labels = response[:100]
test_labels = response[100:]
len(response)

180

In [131]:
from sklearn import linear_model
lr = linear_model.ElasticNet(alpha=0.01)

In [132]:
fit = lr.fit(X=training_data, y=training_labels)

In [133]:
test_predict = fit.predict(test_data)

In [134]:
print(sorted(list(zip(sorted_genes, fit.coef_)), key=lambda x: x[1]))

[('JAK2', -0.06649940677487289), ('MAPK14', -0.025780620852344047), ('MAP4K2', -0.025496499107638036), ('DDR1', -0.018653720827387563), ('CDKL1', -0.018547743693514744), ('RIOK2', -0.014601847023778867), ('AURKB', -0.012973561869730648), ('MARK1', -0.00448997198803679), ('MAP4K5', -0.004106251254130173), ('EPHA8', -0.0018906836047952568), ('AKT3', -0.0012596445522646395), ('AAK1', -0.0), ('ABL1', -0.0), ('ABL2', -0.0), ('ACVR1', 0.0), ('ACVR1B', 0.0), ('ACVR2A', 0.0), ('ACVR2B', -0.0), ('ACVRL1', 0.0), ('ADCK4', 0.0), ('ADRBK1', -0.0), ('ADRBK2', -0.0), ('AKT1', -0.0), ('AKT2', -0.0), ('ALK', 0.0), ('ANKK1', 0.0), ('AURKA', 0.0), ('AURKC', -0.0), ('AXL', 0.0), ('BLK', 0.0), ('BMP2K', -0.0), ('BMPR1A', 0.0), ('BMPR1B', -0.0), ('BMPR2', 0.0), ('BMX', -0.0), ('BRAF', -0.0), ('BRSK1', 0.0), ('BRSK2', 0.0), ('BTK', 0.0), ('BUB1', 0.0), ('CABC1', 0.0), ('CAMK1', -0.0), ('CAMK1D', -0.0), ('CAMK1G', -0.0), ('CAMK2A', 0.0), ('CAMK2B', -0.0), ('CAMK2D', -0.0), ('CAMK2G', -0.0), ('CAMK4', 0.0), (

In [135]:
gene_coefs = dict(zip(sorted_genes, fit.coef_))

In [136]:
gene_coefs

{'AAK1': -0.0,
 'ABL1': -0.0,
 'ABL2': -0.0,
 'ACVR1': 0.0,
 'ACVR1B': 0.0,
 'ACVR2A': 0.0,
 'ACVR2B': -0.0,
 'ACVRL1': 0.0,
 'ADCK4': 0.0,
 'ADRBK1': -0.0,
 'ADRBK2': -0.0,
 'AKT1': -0.0,
 'AKT2': -0.0,
 'AKT3': -0.0012596445522646395,
 'ALK': 0.0,
 'ANKK1': 0.0,
 'AURKA': 0.0,
 'AURKB': -0.012973561869730648,
 'AURKC': -0.0,
 'AXL': 0.0,
 'BLK': 0.0,
 'BMP2K': -0.0,
 'BMPR1A': 0.0,
 'BMPR1B': -0.0,
 'BMPR2': 0.0,
 'BMX': -0.0,
 'BRAF': -0.0,
 'BRSK1': 0.0,
 'BRSK2': 0.0,
 'BTK': 0.0,
 'BUB1': 0.0,
 'CABC1': 0.0,
 'CAMK1': -0.0,
 'CAMK1D': -0.0,
 'CAMK1G': -0.0,
 'CAMK2A': 0.0,
 'CAMK2B': -0.0,
 'CAMK2D': -0.0,
 'CAMK2G': -0.0,
 'CAMK4': 0.0,
 'CAMKK1': 0.0,
 'CAMKK2': 0.0,
 'CASK': -0.0,
 'CDC2L2': 0.0,
 'CDC42BPA': -0.0,
 'CDC42BPB': -0.0,
 'CDC42BPG': -0.0,
 'CDK11B': 0.021771870749973023,
 'CDK13': 0.0,
 'CDK14': 0.0,
 'CDK15': -0.0,
 'CDK16': -0.0,
 'CDK17': -0.0,
 'CDK18': -0.0,
 'CDK19': 0.0,
 'CDK2': 0.0,
 'CDK3': 0.0,
 'CDK4': 0.0,
 'CDK5': 0.0,
 'CDK7': -0.0,
 'CDK8': 0.0,
 